In [1]:
import pandas as pd

In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


### Basic text search
#### using keyword filtering

In [5]:
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


Vector Spaces

- turn the document in vectors
- term-document matrix
    - rows: documents
    - columns: words/tokens
- Bag of words
    - word order is lost
    - sparse matrix
    - (This representation is called "bag of words" - here we ignore the order of words, just focus on the words themselves. In many cases this is sufficient and gives pretty good results already.)

In [6]:
docs_example = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer()

In [9]:
cv.fit(docs_example)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [10]:
names = cv.get_feature_names_out()
names

array(['15th', '2024', 'after', 'and', 'before', 'cloud', 'course',
       'date', 'for', 'github', 'google', 'homeworks', 'jan', 'listed',
       'not', 'on', 'participation', 'prerequisites', 'python',
       'registration', 'required', 'setup', 'start', 'starts', 'submit'],
      dtype=object)

In [11]:
X = cv.transform(docs_example)

In [12]:
X.toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0]])

In [13]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
after,0,0,1,0,0
and,0,0,0,0,1
before,0,0,0,0,1
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
for,0,0,0,1,0
github,0,1,0,0,0


In [14]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


In [15]:
# CountVectorizer turns text into vector
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


TF-IDF (Term Frequency-Inverse Document Frequency)
- Term Frequency (TF): This measures how frequently a term (word) appears in a document.
- Inverse Document Frequency (IDF): This measures how important a term is by considering how often it appears across all documents in the dataset. The more documents a term appears in, the less important it is. 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [17]:
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4
15th,0.46,0.00,0.0,0.00,0.00
2024,0.46,0.00,0.0,0.00,0.00
cloud,0.00,0.00,0.0,0.00,0.46
course,0.37,0.00,0.0,0.00,0.37
date,0.00,0.00,0.5,0.00,0.00
github,0.00,0.58,0.0,0.00,0.00
google,0.00,0.00,0.0,0.00,0.46
homeworks,0.00,0.00,0.5,0.00,0.00
jan,0.46,0.00,0.0,0.00,0.00
listed,0.00,0.58,0.0,0.00,0.00


Query-Document Similarity
We represent the query in the same vector space - i.e. using the same vectorizer:

In [18]:
query = "Do I need to know python to sign up for the January course?"

q = cv.transform([query])
q.toarray()

array([[0.        , 0.        , 0.        , 0.62791376, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.77828292, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

We can see the words of the query and the words of some document:

In [19]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

{'15th': np.float64(0.0),
 '2024': np.float64(0.0),
 'cloud': np.float64(0.0),
 'course': np.float64(0.6279137616509933),
 'date': np.float64(0.0),
 'github': np.float64(0.0),
 'google': np.float64(0.0),
 'homeworks': np.float64(0.0),
 'jan': np.float64(0.0),
 'listed': np.float64(0.0),
 'participation': np.float64(0.0),
 'prerequisites': np.float64(0.0),
 'python': np.float64(0.7782829228046183),
 'registration': np.float64(0.0),
 'required': np.float64(0.0),
 'setup': np.float64(0.0),
 'start': np.float64(0.0),
 'starts': np.float64(0.0),
 'submit': np.float64(0.0)}

In [20]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

{'15th': np.float64(0.0),
 '2024': np.float64(0.0),
 'cloud': np.float64(0.0),
 'course': np.float64(0.0),
 'date': np.float64(0.0),
 'github': np.float64(0.5773502691896258),
 'google': np.float64(0.0),
 'homeworks': np.float64(0.0),
 'jan': np.float64(0.0),
 'listed': np.float64(0.5773502691896258),
 'participation': np.float64(0.0),
 'prerequisites': np.float64(0.5773502691896258),
 'python': np.float64(0.0),
 'registration': np.float64(0.0),
 'required': np.float64(0.0),
 'setup': np.float64(0.0),
 'start': np.float64(0.0),
 'starts': np.float64(0.0),
 'submit': np.float64(0.0)}

The more words in common - the better the matching score. Let's calculate it:

In [21]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

In [22]:
(df_qd['query'] * df_qd['doc']).sum()

np.float64(0.0)

This is a dot-product. So we can use matrix multiplication to compute the score:

In [23]:
X.dot(q.T).toarray()

array([[0.23490553],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59579005]])

In practice, we usually use cosine similarity:

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X, q)

array([[0.23490553],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59579005]])

Vectorizing all the documents
Let's now do it for all the documents:

In [25]:
df.columns

Index(['course', 'section', 'question', 'text'], dtype='object')

In [26]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

In [27]:
transformers['text'].get_feature_names_out()

array(['001', '01', '02', ..., 'zones', 'zoom', 'zoomcamp'],
      shape=(2118,), dtype=object)

In [28]:
matrices['text']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 26463 stored elements and shape (948, 2118)>

#### Search
Let's now do search with the text field:



In [29]:
query = "I just singned up. Is it too late to join the course?"

In [30]:
q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

Let's do it only for the data engineering course:

In [31]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score[:10]

array([0.3336047 , 0.        , 0.        , 0.1328874 , 0.        ,
       0.        , 0.        , 0.12722114, 0.        , 0.        ])

And get the top results:

In [32]:
import numpy as np

In [33]:
idx = np.argsort(-score)[:10]
idx

array([  0,  15,  22,  27,  38, 287,   3,   7, 113,  11])

Note: np.argpartition is a more efficient way of doing the same thing

Get the docs:

In [34]:
score[idx]

array([0.3336047 , 0.23530268, 0.22668   , 0.1894954 , 0.16484429,
       0.13921764, 0.1328874 , 0.12722114, 0.1207499 , 0.10830554])

In [35]:
df.iloc[idx].text

0      The purpose of this document is to capture fre...
15     No, late submissions are not allowed. But if t...
22     It's up to you which platform and environment ...
27     You can do most of the course without a cloud....
38     You will have two attempts for a project. If t...
287    This error could result if you are using some ...
3      You don't need it. You're accepted. You can al...
7      Yes, we will keep all the materials after the ...
113    In the join queries, if we mention the column ...
11     No, you can only get a certificate if you fini...
Name: text, dtype: object

In [36]:
fields

['section', 'question', 'text']

#### Search with all the fields & boosting + filtering
We can do it for all the fields. Let's also boost one of the fields - question - to give it more importance than to others

In [37]:
query = "I just signed up. Is it too late to join the course?"

In [38]:
boost = {'question': 3.0}

score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s

And add filters (in this case, only one):

In [39]:
filters = {
    'course': 'data-engineering-zoomcamp'
}

for field, value in filters.items():
    mask = (df[field] == value).values
    score = score * mask

Getting the results:

In [40]:
idx = np.argsort(-score)[:10]
results = df.iloc[idx]
results.to_dict(orient='records')

[{'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - Which playlist on YouTube should I refer to?',
  'text': 'All the main videos are stored in the Main “DATA ENGINEERING” playlist (no year specified). The Github repository has also been updated to show each video with a thumbnail, that would bring you directly to the same playlist below.\nBelow is the MAIN PLAYLIST’. And then you refer to the

#### Putting it all together
Let's create a class for us to use:

In [41]:
class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')

In [42]:
fields

['section', 'question', 'text']

##### Using it:

In [43]:
index = TextSearch(text_fields=['section', 'question', 'text'])

In [44]:
index.fit(documents)

In [45]:
query

'I just signed up. Is it too late to join the course?'

In [46]:
index.search(
    query='I just signed up. Is it too late to join the course?',
    n_results=5,
    boost={'question': 3.0},
    filters={'course': 'data-engineering-zoomcamp'}
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

## Embeddings and Vector Search
#### Problem with text - only exact matches. How about synonyms?

##### What are Embeddings?
Conversion to Numbers: Embeddings transform different words, sentences and documents into dense vectors (arrays with numbers). 

Capturing Similarity: They ensure similar items have similar numerical vectors, illustrating their closeness in terms of characteristics.
Dimensionality Reduction: Embeddings reduce complex characteristics into vectors.

Use in Machine Learning: These numerical vectors are used in machine learning models for tasks such as recommendations, text analysis, and pattern recognition.

## SVD
Singular Value Decomposition is the simplest way to turn Bag-of-Words representation into embeddings

This way we still don't preserve the word order (because it wasn't in the Bag-of-Words representation) but we reduce dimensionality and capture synonyms.

We won't go into mathematics, it's sufficient to know that SVD "compresses" our input vectors in such a way that as much as possible of the original information is retained.

This compression is lossy compression - meaning that we won't be able to restore the 100% of the original vector, but the result is close enough.

In [47]:
from sklearn.decomposition import TruncatedSVD

In [48]:
X = matrices['text']
cv = transformers['text']

In [49]:
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

In [50]:
X_emb[0]

array([ 0.08800343, -0.07498827, -0.1009309 ,  0.05116636,  0.05363655,
        0.05575623,  0.02081058,  0.05784193, -0.18989205,  0.34540531,
        0.01458636,  0.08344486, -0.12993852,  0.00241388,  0.03340289,
       -0.03658696])

For query:

In [51]:
query = 'I just signed up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)

In [52]:
Q_emb[0]

array([ 4.35377283e-02, -3.05471213e-02, -4.42641706e-02,  1.18982453e-02,
        2.61719060e-02,  4.91724020e-02,  1.31815054e-02,  3.29180372e-02,
       -1.11577353e-01,  1.89086628e-01,  2.00519309e-02,  6.59777875e-02,
       -7.49615402e-02, -1.02397716e-04,  1.36940897e-02, -2.40681122e-02])

Similarity between query and the document:

In [53]:
np.dot(X_emb[0], Q_emb[0])

np.float64(0.1208976687655482)

Let's do it for all the documents. It's the same as previously, except we do it on embeddings, not on sparse matrices:

In [54]:
score = cosine_similarity(X_emb, Q_emb).flatten()

In [55]:
idx = np.argsort(-score)[:10]

In [56]:
list(df.loc[idx].text)

['Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'If you have submitted two projects (and peer-reviewed at least 3 course-mates’ projects for each submission), you will get the certificate for the course. According to the course coordinator, Alexey Grigorev, only two projects are needed to get the course certificate.\n(optional) David Odimegwu',
 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on

## Non-Negative Matrix Factorization
SVD creates values with negative numbers. It's difficult to interpet them.

NMF (Non-Negative Matrix Factorization) is a similar concept, except for non-negative input matrices it produces non-negative results.

We can interpret each of the columns (features) of the embeddings as different topic/concepts and to what extent this document is about this concept.

Let's use it for the documents:

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
X_emb[0]

And the query:

In [ ]:
Q = cv.transform([query])
Q_emb = nmf.transform(Q)
Q_emb[0]